In [ ]:
# imports
import typing
import os
import math
import tensorflow as tf
from tensorflow import keras
import numpy as np
from numpy import genfromtxt
from numpy.lib import recfunctions as rfn
import pandas as pd
import sklearn
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display, Math, Latex
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# colab config
#from google.colab import drive
#drive.mount('/content/drive')
#os.chdir('/content/drive/My Drive/cs470')

# Data Preprocessing

## Data Loading

The dataset was downloaded from Kaggle. It contains 32 Columns of 1,000,000 entries. Since it is 216.59 MB, it may take time to load.

_in this project, "label" will refer to column categories, and "data_actual" will refer to the actual value of the data_

In [ ]:
data = pd.read_csv("Base.csv") # load csv from drive
classification = data.pop("fraud_bool") # remove classification label from dataset
labels = data.keys() # get column labels

# Data Analysis
## Balance check
Our dataset is extremely imbalanced. This is due to the fact that fraudulence overwhelmingly rare in the dataset.

In [ ]:
Latex(f'In the complete data set, {classification.sum()} are fraud ({100*classification.sum()/len(classification):.3f}%)')

# One-hot vectorization

Machine learning algorithms have a hard time understanding strings. To confront this, we replace a column of multiple unique string values with multiple columns for each unique category. These columns contain a boolean to indicate which category it was. This is alternatively known as a _One-hot_.

In [ ]:
# one-hotify labels
labels_categorical = ["payment_type", "employment_status", "housing_status", "source", "device_os"] # list that contains columns to be binarized
labels_vectorized = [] # list that contains all newly created binarized columns
for label in labels_categorical:
    label_index = data.columns.get_loc(label)
    column_binarized = pd.get_dummies(data[label])
    for label_binarized in column_binarized:
        # new label joins the category with the original column name
        label_binarized_new = label + "_" + label_binarized
        data.insert(
            label_index,
            label_binarized_new,
            column_binarized[label_binarized])
        labels_vectorized.append(label_binarized_new)
    del data[label]
# prove that removal occurred and that new columns were added
assert("payment_type" not in data.columns and "device_os" not in data.columns and "device_os_windows" in data.columns)

# Minimum-Maximum Normalization

The backpropagation algorithm will eventually attempt to normalize the range of values within a column to become from 0 to 1. We can save time by preprocessing the data beforehand, shaving seconds if not minutes off of training.

In [ ]:
# minimum-maximum normalization
scaler = MinMaxScaler() # scales data between 0 and 1
for label in labels:
    if label not in labels_categorical: # check only the ones that were not categorical (this implies columns of numbers, not strings)
        data[label] = scaler.fit_transform(data[[label]])

# Data Partitioning

Finally, the data will be partitioned into both training and evaluation subsets.

In [144]:
# data partitioning
training_ratio = 0.85 # this percent will be used for training
training_index = math.floor(len(data)*training_ratio)
data_training = (data[:training_index]) # training data
labels_training = pd.DataFrame(classification[:training_index]) # training data's actual value
print(f"Training data: {len(data_training)}")
display(data_training)
print(f"{data_training.loc[3]}")
data_evaluation = (data[training_index:]) # evaluation data
labels_evaluation  = pd.DataFrame(classification[training_index:]) # evaluation data's actual value

Training data: 850000


,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type_AE,payment_type_AD,payment_type_AC,...,session_length_in_minutes,device_os_x11,device_os_windows,device_os_other,device_os_macintosh,device_os_linux,keep_alive_session,device_distinct_emails_8w,device_fraud_count,month
0,1.000,0.166827,0.000000,0.207459,0.500,0.000267,0.110510,0,0,0,...,0.056250,0,1,0,0,0,0.0,0.666667,0.0,1.000000
1,1.000,0.296285,0.000000,0.337995,0.500,0.000069,0.114520,0,0,0,...,0.377435,0,1,0,0,0,0.0,0.666667,0.0,1.000000
2,1.000,0.044984,0.000000,0.310023,0.375,0.039621,0.114990,0,0,1,...,0.065924,0,0,1,0,0,0.0,0.666667,0.0,1.000000
3,1.000,0.159510,0.000000,0.053613,0.500,0.000243,0.111493,0,0,0,...,0.035074,0,0,0,0,1,0.0,0.666667,0.0,1.000000
4,1.000,0.596414,0.000000,0.510490,0.500,0.000057,0.114854,0,0,0,...,0.054951,0,0,0,1,0,1.0,0.666667,0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849995,0.250,0.742989,0.000000,0.165501,0.500,0.000006,0.111318,0,0,1,...,0.093478,0,1,0,0,0,1.0,0.666667,0.0,0.714286
849996,0.500,0.743531,0.484375,0.018648,0.375,0.000212,0.107038,0,0,0,...,0.081509,0,0,0,0,1,1.0,0.666667,0.0,0.714286
849997,0.000,0.844137,0.000000,0.214452,0.125,0.000135,0.113497,0,0,1,...,0.053083,0,0,1,0,0,1.0,0.666667,0.0,0.714286
849998,0.000,0.242583,0.000000,0.097902,0.250,0.000144,0.115254,0,1,0,...,0.116559,0,0,0,0,1,1.0,0.666667,0.0,0.714286


income                              1.000000
name_email_similarity               0.159510
prev_address_months_count           0.000000
current_address_months_count        0.053613
customer_age                        0.500000
days_since_request                  0.000243
intended_balcon_amount              0.111493
payment_type_AE                     0.000000
payment_type_AD                     0.000000
payment_type_AC                     0.000000
payment_type_AB                     1.000000
payment_type_AA                     0.000000
zip_count_4w                        0.120764
velocity_6h                         0.214831
velocity_24h                        0.335657
velocity_4w                         0.047137
bank_branch_count_8w                0.805451
date_of_birth_distinct_emails_4w    0.153846
employment_status_CG                0.000000
employment_status_CF                0.000000
employment_status_CE                0.000000
employment_status_CD                0.000000
employment

## Class Imbalance
Calculate percentage of training dataset that is true and false respectively.

In [ ]:
weight_for_1 = labels_training.sum() / len(labels_training)
weight_for_0 = 1 - weight_for_1
class_weight = {0: weight_for_0, 1: weight_for_1}
print("weight for 0:",weight_for_0)
print("weight for 1:",weight_for_1)

# Fraud Distribution

In [ ]:
print(labels_training['fraud_bool'].shape)

In [134]:
# # plots distribution between two columns
# def distribution(label_0, label_1):
#     print(labels_training.shape)
#     fraud = pd.DataFrame(columns=data_training.columns)
#     fraud_not = pd.DataFrame(columns=data_training.columns)
#     for row in data_training:
#         if labels_training[row]:
#             fraud.append(row)
#         else:
#             fraud_not.append(row)

#     sns.jointplot(x=fraud[label_0], y=fraud[label_1],
#                 kind='hex', xlim=(-5,5), ylim=(-5,5))
#     plt.suptitle("Fraudulent distribution")

#     sns.jointplot(x=fraud_not[label_0], y=fraud_not[label_1],
#                 kind='hex', xlim=(-5,5), ylim=(-5,5))
#     _ = plt.suptitle("Non-fraudulent distribution")

# distribution("income", "name_email_similarity")

# Model Design

We will be using a simple Multilayer Neural Network for this project. This is because our data is simply just numbers, which have no spatial significance (like an image), so we will not be using a Convolutional Neural Network.

In [120]:
# model design

model = keras.Sequential(
    [
        keras.layers.Dense(256, activation="relu", input_shape=(data_training.shape[-1],)),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)

# Model Compilation

In [121]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.TruePositives(), tf.keras.metrics.AUC(curve="PR", num_thresholds=50)])
model.build()
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 256)               13824     
                                                                 
 dense_17 (Dense)            (None, 256)               65792     
                                                                 
 dropout_8 (Dropout)         (None, 256)               0         
                                                                 
 dense_18 (Dense)            (None, 256)               65792     
                                                                 
 dropout_9 (Dropout)         (None, 256)               0         
                                                                 
 dense_19 (Dense)            (None, 1)                 257       
                                                                 
Total params: 145,665
Trainable params: 145,665
Non-tr

# Model Training

In [122]:
history = model.fit(
    x=data_training,
    y=labels_training,
    class_weight=class_weight,
    epochs=3)

Epoch 1/3
26563/26563 [==============================] - 67s 2ms/step - loss: 9.8056e-05 - binary_accuracy: 0.9998 - precision_6: 0.9983 - recall_6: 0.9811 - false_positives_6: 16.0000 - false_negatives_6: 181.0000 - true_negatives_6: 840407.0000 - true_positives_6: 9396.0000 - auc_1: 0.9879
Epoch 2/3
26563/26563 [==============================] - 63s 2ms/step - loss: 7.7910e-11 - binary_accuracy: 1.0000 - precision_6: 1.0000 - recall_6: 1.0000 - false_positives_6: 0.0000e+00 - false_negatives_6: 0.0000e+00 - true_negatives_6: 840423.0000 - true_positives_6: 9577.0000 - auc_1: 1.0000
Epoch 3/3
26563/26563 [==============================] - 67s 3ms/step - loss: 9.3510e-12 - binary_accuracy: 1.0000 - precision_6: 1.0000 - recall_6: 1.0000 - false_positives_6: 0.0000e+00 - false_negatives_6: 0.0000e+00 - true_negatives_6: 840423.0000 - true_positives_6: 9577.0000 - auc_1: 1.0000


In [ ]:
history = model.evaluate(data_evaluation, labels_evaluation)

4688/4688 [==============================] - 6s 1ms/step - loss: 0.0424 - binary_accuracy: 0.9903 - precision: 0.3810 - recall: 0.0055 - false_positives: 13.0000 - false_negatives: 1444.0000 - true_negatives: 148535.0000 - true_positives: 8.0000


In [147]:
model.predict(np.array([data_evaluation.iloc[5]]))

ValueError: in user code:

    File "/home/csguest/.local/lib/python3.8/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/home/csguest/.local/lib/python3.8/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/csguest/.local/lib/python3.8/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/home/csguest/.local/lib/python3.8/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/home/csguest/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/csguest/.local/lib/python3.8/site-packages/keras/engine/input_spec.py", line 295, in assert_input_compatibility
        ra

In [148]:
print(data_evaluation.shape)

(150000, 52)
